In [1]:
import pyfits
import numpy as np
from astropy.coordinates import SkyCoord, get_moon, EarthLocation, ICRS, GCRS,AltAz
from astropy.time import Time
from astropy.io.fits.hdu.hdulist import HDUList
from datetime import date
from astral import Astral, Location
from astropy.time import Time
import astropy.units as u

In [180]:
import pandas as pd
from itertools import chain
from collections import Counter

In [3]:
## Finding out which file years to download
all_sky=pd.read_csv('objSKY.csv')
all_sky.head()
print('earliest date=', min(all_sky['MJD']), 'or 12-11-2009')
print('latest date=', max(all_sky['MJD']), 'or 05-12-2016')

earliest date= 55176 or 12-11-2009
latest date= 57520 or 05-12-2016


## Data

In [59]:
## To check for reading in the right file
rand_files=('spec-3586-55181-0496.fits','spec-3586-55181-0788.fits','spec-3586-55181-0996.fits',
            'spec-3761-55272-0008.fits','spec-10000-57346-0334.fits','spec-3761-55272-0475.fits',
            'spec-10000-57346-0659.fits','spec-5478-56014-0654.fits','spec-10000-57346-0955.fits',
            'spec-5478-56014-0716.fits','spec-10000-57346-0994.fits')

## To look for patterns
inseq_files=('spec-3663-55176-0010.fits','spec-3663-55176-0012.fits','spec-3663-55176-0020.fits',
             'spec-3663-55176-0024.fits','spec-3663-55176-0036.fits','spec-3663-55176-0038.fits',
             'spec-3663-55176-0048.fits','spec-3663-55176-0052.fits','spec-3663-55176-0056.fits',
             'spec-3663-55176-0068.fits','spec-3663-55176-0075.fits','spec-3663-55176-0078.fits',
             'spec-3663-55176-0090.fits','spec-3663-55176-0094.fits','spec-3663-55176-0096.fits',
             'spec-3663-55176-0108.fits','spec-3663-55176-0112.fits','spec-3663-55176-0114.fits',
             'spec-3663-55176-0128.fits','spec-3663-55176-0134.fits')

solar_files=('g2009.txt','g2010.txt','g2011.txt','g2012.txt','g2013.txt','g2014.txt','g2015.txt','g2016.txt')

## Getting MJDs from fits

In [48]:
#a) first load all files and convert all dates into mjds.
def get_mjd(file):
    da=pyfits.open(file)
    return (da[4].header['MJD'])

In [50]:
for i in rand_files:
    print(get_mjd(i))

55181
55181
55181
55272
57345
55272
57345
56014
57345
56014
57345


## Organizing by date

In [6]:
## Function to organize mill. of sol. disk by date
def organize_by_date(txt_file):
    ## We only need year, month, day of month, and mill. of sol. disk
    ## Cols         1-4    5-6      7-8               31-34
    demo=list(chain.from_iterable((x[:4], x[4:6], x[6:10], x[31:34]) for x in open(txt_file).readlines()))
    ## Organizing cols
    str_list=[demo[i*4:i*4+4] for i in range(int(len(demo)/4))]
    ## Converting to float
    float_dates=[]
    for k in range(len(str_list)):
        float_dates.append([float(i) for i in str_list[k]])
    return float_dates

In [88]:
#final_list=organize_by_date('g2009.txt')
#final_list

In [63]:
#for i in solar_files:
#    print(organize_by_date(i))

## Interpolated function from MJDs to mill of sol. disk

In [226]:
#b) Then set up one massive interpolated function from mjds to mill of sol. disk
## Pulling out cols of interest
demo=list(chain.from_iterable((x[:4], x[4:6], x[6:10], x[31:34]) for x in open('g2011.txt').readlines()))
## Organizing cols
str_list=[demo[i*4:i*4+4] for i in range(int(len(demo)/4))]
## Converting to float
float_dates=[]
for k in range(len(str_list)):
    float_dates.append([float(i) for i in str_list[k]])
#    if [x[:3] for x in float_dates][k] == [x[:3] for x in float_dates][k+1]:
#        print([x[:3] for x in float_dates][k], [x[:3] for x in float_dates][k-1])
        #print(np.mean([x[3] for x in float_dates][k], [x[:3] for x in float_dates][k-1]))
        #print([x[:4] for x in float_dates][k]),([x[:4] for x in float_dates][k-1])
        #print([x[:3] for x in float_dates][k] and [x[:3] for x in float_dates][k-1])
        #print([x[:3] for x in float_dates])

## Finding the duplicates, averaging mill of sol. disk, remove duplicate rows

In [229]:
#[x[:3] for x in float_dates]
#[i for i, x in ([x[:3] for x in float_dates]) if [x[:3] for x in float_dates].count(x) > 1]
print(len(str_list))
print(len(float_dates))

1791
1791


In [232]:
mod_solar_dates=[]
for i in range(len(str_list)):
    if ([x[:3] for x in float_dates][i]) == ([x[:3] for x in float_dates][i+1]) or ([x[:3] for x in float_dates][i]) == ([x[:3] for x in float_dates][i-1]):
        #mod_solar_dates.append([x[:4] for x in float_dates][i])
        print([x[:4] for x in float_dates][i])
    else:
        print(i+1)
        #print([x[:4] for x in float_dates][i])
        #print([sum(y) / len(y) for y in zip(*[x[3] for x in float_dates][i])])
        #print([x[:4] for x in float_dates][i])

[2011.0, 1.0, 1.5, 2.0]
[2011.0, 1.0, 1.5, 0.0]
[2011.0, 1.0, 1.5, 196.0]
[2011.0, 1.0, 1.5, 141.0]
[2011.0, 1.0, 1.5, 66.0]
[2011.0, 1.0, 2.5, 197.0]
[2011.0, 1.0, 2.5, 12.0]
[2011.0, 1.0, 2.5, 166.0]
[2011.0, 1.0, 3.5, 264.0]
[2011.0, 1.0, 3.5, 19.0]
[2011.0, 1.0, 3.5, 234.0]
[2011.0, 1.0, 4.5, 322.0]
[2011.0, 1.0, 4.5, 38.0]
[2011.0, 1.0, 4.5, 177.0]
[2011.0, 1.0, 5.5, 300.0]
[2011.0, 1.0, 5.5, 6.0]
[2011.0, 1.0, 5.5, 75.0]
[2011.0, 1.0, 6.5, 282.0]
[2011.0, 1.0, 6.5, 83.0]
[2011.0, 1.0, 7.5, 5.0]
[2011.0, 1.0, 7.5, 203.0]
[2011.0, 1.0, 7.5, 0.0]
[2011.0, 1.0, 7.5, 36.0]
[2011.0, 1.0, 8.5, 0.0]
[2011.0, 1.0, 8.5, 204.0]
[2011.0, 1.0, 8.5, 11.0]
[2011.0, 1.0, 8.5, 18.0]
[2011.0, 1.0, 9.5, 134.0]
[2011.0, 1.0, 9.5, 5.0]
[2011.0, 1.0, 9.5, 0.0]
[2011.0, 1.0, 9.5, 14.0]
[2011.0, 1.0, 10.5, 96.0]
[2011.0, 1.0, 10.5, 16.0]
[2011.0, 1.0, 10.5, 0.0]
[2011.0, 1.0, 11.5, 45.0]
[2011.0, 1.0, 11.5, 14.0]
[2011.0, 1.0, 12.5, 0.0]
[2011.0, 1.0, 12.5, 16.0]
39
40
41
42
[2011.0, 1.0, 17.5, 156.0]
[

IndexError: list index out of range

In [231]:
## This averaged everything into one tuple. We are trying to average the duplicates... 
[sum(y) / len(y) for y in zip(*mod_solar_dates)]

[2011.0, 7.672849915682968, 16.077852726250704, 132.60146149522203]